# Projeto em Análise de Dados

### Importando as bibliotecas necessárias para o projeto

In [59]:
import os
import os.path
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta


### 1) Tratamento de Dados - ETL

#### A parte "E" (extract) do ETL já foi feita de antemão, uma vez que os dados do drive com as planilhas mockadas foram baixados e transferidos para esta pasta. Os próximos passos são fazer a leitura dos dados e transformação dos mesmos para posteriormente fazermos o LOAD em um DataFrame do Pandas.

In [60]:
## Inicializo o DataFrame em Pandas:

df = pd.DataFrame()
dfs = [] # Vetor fora do Pandas para receber appends


caminho = 'Tabelas/'
lista_arquivos = os.listdir(caminho)

# For para varrer toda a pasta de planilhas mockadas:
for arquivo in lista_arquivos:
    df_aux = pd.read_csv(caminho+str(arquivo))
    dfs.append(df_aux)

df = pd.concat(dfs, ignore_index=True) # Concatenação de dataframes

df
    

,ano,uf,genero,mes,numero
0,2019,Rondônia,Masculino,Janeiro,0
1,2019,Rondônia,Masculino,Fevereiro,4
2,2019,Rondônia,Masculino,Março,1
3,2019,Rondônia,Masculino,Abril,0
4,2019,Rondônia,Masculino,Maio,0
...,...,...,...,...,...
4531,2021,Distrito Federal,Feminino,Agosto,12
4532,2021,Distrito Federal,Feminino,Setembro,13
4533,2021,Distrito Federal,Feminino,Outubro,15
4534,2021,Distrito Federal,Feminino,Novembro,20


#### Agora que temos um DataFrame único, podemos fazer o tratamento de dados, pois alguns campos (colunas) ainda estão com o type de 'Object'

In [61]:
df.dtypes

ano        int64
uf        object
genero    object
mes       object
numero     int64
dtype: object

#### Não pertinente dexar duas colunas de data (como por exemplo, ano e mês) separadas, sendo assim podemos juntá-las em uma única e transformar os dados em Date Time

In [62]:
#Criação de um dicionário para mapear os meses e transformá-los em números.

dicionario_mes = {
    'Janeiro': '01',
    'Fevereiro': '02',
    'Março': '03',
    'Abril': '04',
    'Maio': '05',
    'Junho': '06',
    'Julho': '07',
    'Agosto': '08',
    'Setembro': '09',
    'Outubro': '10',
    'Novembro': '11',
    'Dezembro': '12'
}

In [63]:
#Criação da coluna de numero_mes que corresponde ao número do respectvo mês do dicionário acima.
#Utilizei o método map que maepeia os valores do dicionário
df['numero_mes'] = df['mes'].map(dicionario_mes)
df

,ano,uf,genero,mes,numero,numero_mes
0,2019,Rondônia,Masculino,Janeiro,0,01
1,2019,Rondônia,Masculino,Fevereiro,4,02
2,2019,Rondônia,Masculino,Março,1,03
3,2019,Rondônia,Masculino,Abril,0,04
4,2019,Rondônia,Masculino,Maio,0,05
...,...,...,...,...,...,...
4531,2021,Distrito Federal,Feminino,Agosto,12,08
4532,2021,Distrito Federal,Feminino,Setembro,13,09
4533,2021,Distrito Federal,Feminino,Outubro,15,10
4534,2021,Distrito Federal,Feminino,Novembro,20,11


In [64]:
df.dtypes

ano            int64
uf            object
genero        object
mes           object
numero         int64
numero_mes    object
dtype: object

In [65]:
df['ano'] = df['ano'].astype(str) # Transformar o Ano em string, pois ainda está em int
df['numero_mes'] = df['numero_mes'].astype(str) # Transformar o Ano em string, pois ainda está em object
df['data'] = pd.to_datetime(df['ano'] + '-' + df['numero_mes'] + '-01') # Como o dia do mês não importa na análise, podemos omitir esse campo
# Por default, as datas ficarão no formato aaaa-mm-dd, o campo de dia '-dd' ficará sempre '01'

In [66]:
df

,ano,uf,genero,mes,numero,numero_mes,data
0,2019,Rondônia,Masculino,Janeiro,0,01,2019-01-01
1,2019,Rondônia,Masculino,Fevereiro,4,02,2019-02-01
2,2019,Rondônia,Masculino,Março,1,03,2019-03-01
3,2019,Rondônia,Masculino,Abril,0,04,2019-04-01
4,2019,Rondônia,Masculino,Maio,0,05,2019-05-01
...,...,...,...,...,...,...,...
4531,2021,Distrito Federal,Feminino,Agosto,12,08,2021-08-01
4532,2021,Distrito Federal,Feminino,Setembro,13,09,2021-09-01
4533,2021,Distrito Federal,Feminino,Outubro,15,10,2021-10-01
4534,2021,Distrito Federal,Feminino,Novembro,20,11,2021-11-01


#### Agora podemos 'Dropar' as colunas 'numero_mes', 'mes' e 'ano' para fins de performance, mas podemos manter as mesmas colunas caso sejam pertinentes na análise (neste caso irei manter as colunas ainda).

In [67]:
df.dtypes

ano                   object
uf                    object
genero                object
mes                   object
numero                 int64
numero_mes            object
data          datetime64[ns]
dtype: object